In [1]:
import pandas as pd

# Step 1: Read raw data
df = pd.read_csv("raw_data.csv")

# Step 2: Convert date columns to datetime
date_cols = ["Event date", "Payment date", "Listing created Date"]
df[date_cols] = df[date_cols].apply(pd.to_datetime, errors='coerce')

# Step 3: Convert ID columns to string
id_cols = ["Ticket ID", "Seller user ID", "Buyer user ID"]
df[id_cols] = df[id_cols].astype("string")

# Step 4: Clean and convert currency columns to numeric
price_columns = [
    "Ticket price buyer",
    "Original ticket price",
    "Ticket listing price",
    "Revenue",
    "Kickback",
    "Fairshare"
]

for col in price_columns:
    df[col] = pd.to_numeric(
        df[col].str.replace("€", "").str.replace(",", ""),  # Remove currency symbols and commas
        errors="coerce"
    )

# Step 5: Convert Yes/No column to boolean
df["Meets ticket Fairshare conditions? (Yes / No)"] = (
    df["Meets ticket Fairshare conditions? (Yes / No)"]
    .str.strip().str.lower()
    .map({"yes": True, "no": False})
)

# Step 6: Add Ratio column (Fairshare / Kickback)
df["Ratio"] = df["Fairshare"] / df["Kickback"]

# Step 7: Assign Ticket Class within each Event
def index_to_letters(n: int) -> str:
    """
    Convert number to Excel-style letters: 0 → A, 1 → B, ..., 25 → Z, 26 → AA, etc.
    """
    letters = ""
    while n >= 0:
        letters = chr(65 + (n % 26)) + letters
        n = n // 26 - 1
    return letters

def assign_classes_within_event(group: pd.DataFrame, price_col: str) -> pd.Series:
    """
    Assign ticket classes (A, B, C...) within a single event based on price ranking.
    Equal prices receive the same class (dense ranking).
    """
    price_rank = (
        group[price_col]
        .rank(method="dense", ascending=False)   # Rank high price = A
        .astype("Int64")                         # Keep nullable integers
    )
    return price_rank.map(lambda r: index_to_letters(r - 1) if pd.notna(r) else pd.NA)

# Apply class assignment only if relevant columns exist
if "Original ticket price" in df.columns and "Event" in df.columns:
    df["Ticket_Class"] = (
        df.groupby("Event", group_keys=False)
          .apply(lambda g: assign_classes_within_event(g, "Original ticket price"))
    )

# Step 8: Remove faulty or extreme outlier row
df = df.drop(index=6767)  # Row 6767 had excessive missing or unrealistic values

# Step 9: Export cleaned dataset
df.to_csv("cleaned_data.csv", index=False)
